In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests11 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False
data_tuned_epsilon = True
epsilon = .5

size weirdness rn...

In [5]:
a = 0.25
num_params=9

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.93 & 0.79 & 0.93 & 0.96   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.07 & 0.02 & 0.02   \\
Model 2 & 0.04 & 0.04 & 0.05 & 0.13 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.942, 0.014, 0.044]), array([0.942, 0.014, 0.044]), array([0.93 , 0.016, 0.054]), array([0.794, 0.074, 0.132]), array([0.932, 0.018, 0.05 ]), array([0.962, 0.024, 0.014]), -3.3969762942471795, 11.105464124481172, 11.031226649035414)


# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.85 & 0.88 & 0.75 & 0.86 & 0.57   \\
Model 1 & 0.16 & 0.15 & 0.12 & 0.22 & 0.14 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.844, 0.156, 0.   ]), array([0.85, 0.15, 0.  ]), array([0.882, 0.116, 0.002]), array([0.752, 0.222, 0.026]), array([0.858, 0.138, 0.004]), array([0.572, 0.428, 0.   ]), 6.51432982135082, 5.151250764724778, 5.333167897050029)


In [8]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.82 & 0.86 & 0.72 & 0.85 & 0.55   \\
Model 1 & 0.18 & 0.18 & 0.14 & 0.26 & 0.15 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.816, 0.184, 0.   ]), array([0.818, 0.182, 0.   ]), array([0.86 , 0.138, 0.002]), array([0.716, 0.258, 0.026]), array([0.85 , 0.146, 0.004]), array([0.548, 0.452, 0.   ]), 6.550707675297704, 5.301940413114315, 5.393158987728192)


In [9]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.82 & 0.83 & 0.86 & 0.75 & 0.85 & 0.55   \\
Model 1 & 0.18 & 0.17 & 0.14 & 0.22 & 0.15 & 0.45   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.03 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.824, 0.176, 0.   ]), array([0.828, 0.172, 0.   ]), array([0.862, 0.138, 0.   ]), array([0.748, 0.224, 0.028]), array([0.85, 0.15, 0.  ]), array([0.552, 0.448, 0.   ]), 6.52020360634654, 4.869970842981383, 5.283739888783443)


In [10]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.84 & 0.84 & 0.88 & 0.73 & 0.87 & 0.57   \\
Model 1 & 0.16 & 0.16 & 0.11 & 0.23 & 0.13 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.04 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.842, 0.158, 0.   ]), array([0.844, 0.156, 0.   ]), array([0.884, 0.112, 0.004]), array([0.73 , 0.226, 0.044]), array([0.866, 0.128, 0.006]), array([0.572, 0.428, 0.   ]), 6.544916377253782, 4.859664496262733, 5.336604424592619)


In [11]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.88 & 0.74 & 0.88 & 0.58   \\
Model 1 & 0.17 & 0.17 & 0.12 & 0.23 & 0.12 & 0.42   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.83, 0.17, 0.  ]), array([0.834, 0.166, 0.   ]), array([0.884, 0.116, 0.   ]), array([0.744, 0.232, 0.024]), array([0.88 , 0.118, 0.002]), array([0.578, 0.422, 0.   ]), 6.631590202325172, 5.290947662910752, 5.317767678929262)


# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.91 & 0.91 & 0.91 & 0.81 & 0.89 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.02 & 0.00 & 0.02   \\
Model 2 & 0.09 & 0.09 & 0.09 & 0.17 & 0.11 & 0.02   \\
\hline
\end{tabular}
(array([0.906, 0.004, 0.09 ]), array([0.908, 0.004, 0.088]), array([0.908, 0.006, 0.086]), array([0.808, 0.02 , 0.172]), array([0.89 , 0.004, 0.106]), array([0.964, 0.02 , 0.016]), -4.070487047553493, 6.034478005337475, 6.004696542220461)


In [14]:
nobs = 500

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.94 & 0.80 & 0.94 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.04 & 0.00 & 0.02   \\
Model 2 & 0.07 & 0.07 & 0.05 & 0.16 & 0.06 & 0.01   \\
\hline
\end{tabular}
(array([0.926, 0.004, 0.07 ]), array([0.926, 0.004, 0.07 ]), array([0.944, 0.002, 0.054]), array([0.796, 0.042, 0.162]), array([0.936, 0.   , 0.064]), array([0.964, 0.022, 0.014]), -4.0957366439623435, 7.57942700769747, 7.962210328731987)


In [15]:
nobs = 100

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.96 & 0.88 & 0.75 & 0.85 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.01   \\
Model 2 & 0.13 & 0.04 & 0.12 & 0.23 & 0.15 & 0.01   \\
\hline
\end{tabular}
(array([0.866, 0.002, 0.132]), array([0.962, 0.   , 0.038]), array([0.884, 0.   , 0.116]), array([0.748, 0.02 , 0.232]), array([0.852, 0.   , 0.148]), array([0.984, 0.01 , 0.006]), -4.1184999863348954, 4.0163290457624985, 4.282900354455957)


# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.93 & 0.83 & 0.92 & 0.97   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.07 & 0.03 & 0.02   \\
Model 2 & 0.03 & 0.03 & 0.05 & 0.11 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.948, 0.018, 0.034]), array([0.948, 0.018, 0.034]), array([0.93 , 0.024, 0.046]), array([0.826, 0.066, 0.108]), array([0.92 , 0.028, 0.052]), array([0.97, 0.02, 0.01]), -1.3471651060868195, 5.878982195705247, 5.615110484735896)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:
nobs = 250

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.74 & 0.81 & 0.80 & 0.73 & 0.79 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.00 & 0.01   \\
Model 2 & 0.26 & 0.19 & 0.20 & 0.25 & 0.21 & 0.01   \\
\hline
\end{tabular}
(array([0.736, 0.   , 0.264]), array([0.814, 0.   , 0.186]), array([0.8, 0. , 0.2]), array([0.732, 0.016, 0.252]), array([0.788, 0.   , 0.212]), array([0.976, 0.014, 0.01 ]), -9.170518358001168, 6.188188948626263, 6.66282969971595)


# evidence of power

In [20]:
nobs = 500

a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.99 & 0.96 & 0.79 & 0.95 & 0.62   \\
Model 1 & 0.01 & 0.01 & 0.03 & 0.13 & 0.03 & 0.38   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.08 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.984, 0.01 , 0.006]), array([0.99 , 0.008, 0.002]), array([0.962, 0.03 , 0.008]), array([0.786, 0.134, 0.08 ]), array([0.954, 0.034, 0.012]), array([0.62, 0.38, 0.  ]), 1.8600869296520524, 5.340265174797508, 6.275632671822521)


In [21]:
nobs = 100

a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 1.00 & 0.95 & 0.89 & 0.94 & 0.71   \\
Model 1 & 0.02 & 0.00 & 0.02 & 0.05 & 0.02 & 0.29   \\
Model 2 & 0.02 & 0.00 & 0.03 & 0.06 & 0.03 & 0.00   \\
\hline
\end{tabular}
(array([0.964, 0.02 , 0.016]), array([0.998, 0.002, 0.   ]), array([0.954, 0.018, 0.028]), array([0.89 , 0.048, 0.062]), array([0.944, 0.022, 0.034]), array([0.706, 0.294, 0.   ]), 0.4163989851554653, 5.572973359457482, 5.810543377268482)


In [22]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.63 & 0.63 & 0.77 & 0.68 & 0.76 & 0.56   \\
Model 1 & 0.37 & 0.37 & 0.23 & 0.31 & 0.24 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.626, 0.374, 0.   ]), array([0.626, 0.374, 0.   ]), array([0.768, 0.23 , 0.002]), array([0.678, 0.31 , 0.012]), array([0.764, 0.236, 0.   ]), array([0.562, 0.438, 0.   ]), 8.667584055803768, 4.699394000313578, 4.873728383160587)


In [23]:
nobs = 250

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.98 & 0.96 & 0.83 & 0.95 & 0.69   \\
Model 1 & 0.03 & 0.02 & 0.03 & 0.09 & 0.03 & 0.31   \\
Model 2 & 0.02 & 0.00 & 0.02 & 0.08 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.956, 0.026, 0.018]), array([0.976, 0.02 , 0.004]), array([0.958, 0.026, 0.016]), array([0.83, 0.09, 0.08]), array([0.946, 0.032, 0.022]), array([0.69, 0.31, 0.  ]), 1.2345125125591876, 5.682888377249747, 6.1827458443001255)


# evidence of power 2

In [24]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.58 & 0.20 & 0.30 & 0.21 & 0.68   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.42 & 0.80 & 0.70 & 0.79 & 0.32   \\
\hline
\end{tabular}
(array([0.008, 0.   , 0.992]), array([0.584, 0.   , 0.416]), array([0.198, 0.   , 0.802]), array([0.298, 0.   , 0.702]), array([0.21, 0.  , 0.79]), array([0.684, 0.   , 0.316]), -19.757961490816715, 5.311475959426946, 6.098938013083327)


In [25]:
nobs = 500


a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.37 & 0.37 & 0.42 & 0.36 & 0.71   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.63 & 0.63 & 0.58 & 0.64 & 0.29   \\
\hline
\end{tabular}
(array([0.004, 0.   , 0.996]), array([0.366, 0.   , 0.634]), array([0.368, 0.   , 0.632]), array([0.42, 0.  , 0.58]), array([0.362, 0.   , 0.638]), array([0.706, 0.   , 0.294]), -19.94202637919931, 5.8172522955837, 6.274747692289874)


In [26]:
nobs = 100


a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.95 & 0.05 & 0.12 & 0.06 & 0.59   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.05 & 0.95 & 0.88 & 0.94 & 0.41   \\
\hline
\end{tabular}
(array([0.004, 0.   , 0.996]), array([0.952, 0.   , 0.048]), array([0.046, 0.   , 0.954]), array([0.12, 0.  , 0.88]), array([0.06, 0.  , 0.94]), array([0.586, 0.   , 0.414]), -20.784432911081513, 5.676402544786004, 5.748212235611171)


In [27]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.25 & 0.25 & 0.54 & 0.51 & 0.53 & 0.56   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.75 & 0.75 & 0.46 & 0.48 & 0.47 & 0.44   \\
\hline
\end{tabular}
(array([0.252, 0.   , 0.748]), array([0.254, 0.   , 0.746]), array([0.544, 0.   , 0.456]), array([0.514, 0.004, 0.482]), array([0.53, 0.  , 0.47]), array([0.562, 0.   , 0.438]), -11.999573247398455, 4.556311150740601, 4.850712696351638)


In [28]:
nobs = 250


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,epsilon=epsilon,
                                      data_tuned_epsilon=data_tuned_epsilon ,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.58 & 0.18 & 0.26 & 0.18 & 0.64   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.42 & 0.82 & 0.74 & 0.82 & 0.36   \\
\hline
\end{tabular}
(array([0.012, 0.   , 0.988]), array([0.58, 0.  , 0.42]), array([0.178, 0.   , 0.822]), array([0.258, 0.   , 0.742]), array([0.18, 0.  , 0.82]), array([0.64, 0.  , 0.36]), -20.12915746805983, 5.470411051232837, 6.124168723679673)
